Note: Write your code in the code cells, and your responses in markdown. 
Run the entire script and display the outputs of your code. Upload both your code (.ipynb) and responses (html or pdf) to Canvas.   

<!-- Helpful tips on Markdown: - https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html# -->

# Simulation - Bernoulli
Set up an Python program to conduct a simulation of drawing a sample size of $n$ from a Bernoulli distribution with mean $p$.  
In each "replication" $r$ you will draw a sample, construct the estimated mean from that replication (which we will denote as $\overline{Y}_r$) , and calculate the 95% confidence interval $(\overline{Y}_r-1.96*s_r/\sqrt{n},\, \overline{Y}_r-1.96*s_r/\sqrt{n})$  where $s_r$ is the estimated standard deviation in that replication, $s_r=\sqrt{\overline{Y}_r*(1-\overline{Y}_r)}$. In each replication, record the length of the confidence interval, and whether or not the true mean is inside the interval.  

For given $(n, p)$, conduct $1,000$ replications are report the following statistics: 

1. the mean estimate of $p$
    - plot the distribution of $\overline{Y}_r$ (sample estimates of $p$) across replications. 
2. the mean estimate of the true standard deviation
3. the fraction of time that the confidence interval contains the true $p$.  This is called the *coverage rate*. 

Conduct the analysis for the cases $n=30$ using $p=0.05$ and $p=0.25$,  and again for $n=60$ using $p=0.05$ and $p=0.25$  (a total of 4 cases). It is often claimed that $n$ of 30 or larger is enough to insure that asymptotic confidence intervals work well. Do you agree or not?

**Answer** 
Recall Binomial$(n,p)$ is defined as $n$ independent Bernoulli trials with mean $p$. Binomial(1,p) is equivalent to Bernoulli(p). 
First, we create a function to run simulations. The inputs are $n$ = sample size in each replication, $p$ = true mean of the Bernoulli distribution, and $R$ = num. replications. 
The second function "hist_phat" will plot the histogram of the estimates for $p$. 

In [ ]:
import numpy as np 
np.random.seed(10)
import matplotlib.pyplot as plt # for plotting 
import warnings
warnings.filterwarnings('ignore')

# function for the Bernoulli simulation: note: Bernoulli(p) is equivalent to Binomial(1,p)
def bernoulli(n,p,R):
    # inputs: n = sample size, p = true mean, R = num. replications 
    # save the sample mean/sd/coverage in each replication: 
    l_mean = []; l_sd = []; l_cover = []
    for r in range(R):
        data = np.random.binomial(1,p,n)    
        # sample mean & sd:  
        y_r, s_r = data.mean(), data.std()
        # is p covered by the confidence interval? 
        ci_low = y_r - 1.96*s_r/np.sqrt(n)
        ci_high = y_r +  1.96*s_r/np.sqrt(n)
        cover_r = 0+ ((p>=ci_low) & (p<=ci_high))
        # append to lists: 
        l_mean.append(y_r); l_sd.append(s_r); l_cover.append(cover_r)
    # output: the list of means & standard deviations from R simulations
    assert (R== len(l_mean)) & (R==len(l_sd))  & (R==len(l_cover))
    return(l_mean, l_sd, l_cover)


Now we run the simulation function for $n\in\{30,60\}$, and $p\in\{0.05,0.25\}$. 

In [ ]:
R = 1000 # num. replications
l_n = [30,60]; l_p = [0.05,0.25]; 

for p in l_p:
    for n in l_n:
        print(f"case: (n={n}, p ={p})")
        l_mean, l_sd, l_cover = bernoulli(n,p,R) 
        print(f"mean estimate of p: {np.array(l_mean).mean()}")
        print(f"mean estimate of sd: {np.array(l_sd).mean()}")
        print(f"coverage: {np.array(l_cover).mean()}")
        # plot the estimates of p:
        plt.figure(); plt.hist(np.array(l_mean), bins=30,alpha=0.25, edgecolor='black'); plt.xlabel(r"$\overline{Y}_r$: estimates of $p$, across 1000 replications"); 
        plt.show()



For true mean $p=0.05$, the coverage rates at $n=30$ or $n=60$ are around 0.80, which are pretty low. 
For true mean $p=0.25$, the coverage rates are higher, above 0.90 even for a small sample like $n=30$. The distribution of $\overline{Y}$ across replications looks approximately normal for either $n=30$ or $n=60$. 

Why is the coverage different for $p=0.05$ vs. 0.25? Think about how the standard deviation converges to the truth: $\hat{\sigma} = \sqrt{n} * \sqrt{p(1 − p)}$. As $p$ gets close to 0 or 1, that will slow down the rate of convergence. This is especially true of small samples (small $n$). This is a known problem with binary random variables – for very rare events, the conventional wisdom of $n = 30$ breaks down. 

# Data analysis: Age Profile and Gender Gap in the Use of Cell Phone
Download the dataset "october_cps", which is an extract of data from the October 2012 CPS.  

There are several variables describing each person in the survey, including age, sex, "educ" (which is education, coded in a certain way) as well as variables about how someone uses their cell phone (if they have one): cell_use_phone, cell_use_msg, cell_use_video, cell_use_browse_web, cell_use_email, cell_use_games, cell_use_social_media, cell_use_download_apps, and cell_use_music.  Each of these is coded "1" if the person uses the cell phone for that use, "2" if not, and  (as a phone; for text messages, for video, to browse the web, for email for games, to access social media, to download apps, or to listen to music). 

1. Develop a graph to show how people of different ages use their cell phone. Be creative. 
2. In your sample, test whether males (sex=1) and females (sex=2) use their cell phone at the same rate for each of the 9 different uses.  

## Age Profile
According to the codebook, the cell_use*variables are coded as 1 if Yes, 2 if No (common practice in administrative surveys), -1 if no cell phone. We replace them by indicators for cell phone use in each task.  

In [ ]:
import pandas as pd 
pd.options.display.max_columns = None
import numpy as np 
# import os 
# os.chdir() # set the directory to where you saved the dataset 

# read data:
oct = pd.read_csv("october_cps.csv")
print(oct.shape, oct.head(2))

# note: according to the codebook, 1= Yes, 2 = No. Convert to indicators instead: 
vars= [x for x in oct.columns if 'cell_use' in x]
for x in vars:
    oct[x] = 0 + (oct[x]==1)


Below is a function for plotting the fraction of individuals using cell phone for each task, against age. The input "lab" is a dictionary that specifies x-variable, y-variable(s), and parameters in the figure. 

In [ ]:
import matplotlib.pyplot as plt 

# define a function for plotting the age profile: 
def plot0(data, lab):  
    # data: has mean(var) sd(var)
    # lab: a dictionary with parameters/labels... for the figure 
    font_size = 11
    x, Y = lab['xvar'], lab['yvar']
    # setup figure
    plt.clf(); plt.close('all') 
    fig, ax1 = plt.subplots(1,1)
    fig.set_size_inches(15, 9)

    # allow multiple lines (for multipe variables) 
    for j in range(len(Y)):
        y = Y[j];  y_legend = lab['legend'][j]; 
        y_lcolor = lab['color'][j];  
        if lab['error_bar']==0: 
            ax1.plot(data[x], data[y], linestyle=lab['line'][j], color= y_lcolor, lw=1.5, alpha=0.95,
                        marker = lab['marker'][j], markersize=lab['msize'], 
                        label = y_legend)  
        else: 
            ax1.errorbar(data[x], data[y], data[f'sd_{y}']*1.96, linestyle='-',color= y_lcolor, lw=1.5, alpha=0.95) # https://matplotlib.org/stable/api/markers_api.html '^'
    ax1.set_xlabel(lab['xlab'], labelpad=10, fontsize=font_size)
    ax1.set_ylabel(lab['ylab'], labelpad=10, fontsize=font_size)
    # legend: 
    lines, labels = ax1.get_legend_handles_labels() 
    ax1.legend(lines,labels, loc=0); plt.legend(fontsize=font_size)
    # xticks
    if 'xticks_lab' in lab.keys():
        ax1.set_xticks(lab['xticks'], labels= lab['xticks_lab'], rotation=45) # https://stackoverflow.com/questions/11244514/modify-tick-label-text
    # title: 
    if 'title' in lab.keys():
        if lab['title']!='':
            fig.suptitle(lab['title'],fontsize=font_size+2)
    # layout, save figure 
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    plt.show()
    # plt.savefig(lab['file'], bbox_inches='tight')
    return()



Now we are ready to plot the age profile. "groupby(k)" helps summarize the data by variable *k*. 

In [ ]:
# Age profile: (mean, sd) of cell-phone use by age: 

# Variables on cell phone use
dict_stats = {x: ['mean','std'] for x in vars}

# 1) by age: 
by_age = oct.groupby(['age']).agg(dict_stats).reset_index(drop=False) 
by_age.columns=[x[0] if x[1]!='std' else 'sd_'+x[0] for x in by_age.columns] 
print(by_age.shape,by_age.head(2))

# create a dictionary for figure specifics: 
lab = {'xvar': 'age', 'yvar': vars, 'color': plt.get_cmap('Dark2', 9).colors, 
       'line':['-']*9,  
        'marker': ['.','s','o','v','*','x','d','h','1'], 'msize':5, 
        'legend': [x.strip('cell_use_') for x in vars], 
        'error_bar':0, 
        'xlab':'Age','ylab':'Fraction who use phone',
        'title': 'Age Profile of the Use of Cell Phone'} 
plot0(by_age, lab) 

Alternatively, we can define age groups first, and then plot the cell phone use by age group. The graph is cleaner. 

In [ ]:
# 2) by age group: 
oct['age_group'] = 1*(oct['age'].between(16,20)) + 2*(oct['age'].between(21,30)) + 3*(oct['age'].between(31,40)) + 4* (oct['age'].between(41,50)) + 5*(oct['age'].between(51,60)) + 6* (oct['age']>=61)
print(oct.groupby(['age_group']).agg({'age':['min','max']})) 
by_age = oct.groupby(['age_group']).agg(dict_stats).reset_index(drop=False) 
by_age.columns=[x[0] if x[1]!='std' else 'sd_'+x[0] for x in by_age.columns] 
print(by_age.shape,by_age.head(2))

lab = {'xvar': 'age_group', 'yvar': vars, 
       'color': plt.get_cmap('Dark2', 9).colors, 
       'line':['-']*9,  
       'marker': ['.','s','o','v','*','x','d','h','1'], 'msize':8,
        'xlab':'Age','ylab':'Fraction who use phone',
        'legend': [x.strip('cell_use_') for x in vars], 
        'error_bar':0, 
        'title': 'Age Profile of the Use of Cell Phone',
        'xticks':np.arange(1,7, step=1) , 
        'xticks_lab': ['16-20', '21-30','31-40','41-50','51-60','>60']
} 
print(lab)
plot0(by_age, lab) 

## Gender Gap
Now we use groupby function again to summarize cell phone use by sex. 

In [ ]:
print(oct['sex'].value_counts(dropna=False))
by_sex = oct.groupby(['sex']).agg(dict_stats).reset_index(drop=False) 
by_sex.columns=[x[0] if x[1]!='std' else 'sd_'+x[0] for x in by_sex.columns] 
n_F = (oct['sex']==2).sum(); 
n_M = (oct['sex']==1).sum() 
print(f'{n_F} females and {n_M} males in sample')

To test if the mean for females and the mean for males are equal, we conduct a two-sample Welch's t-test (allowing for unequal variance). The t statistic is defined by: $$t \coloneqq \frac{\bar{x}_F - \bar{x}_M}{\sqrt{s^2_F/n_F + s^2_M/n_M}}$$ where $\bar{x}_F$ is the sample mean of $x$ among females, and $s^2_F/n_F$ is an estimate of the variance of the mean. 

In [ ]:
by_sex.head()

In [ ]:
a = by_sex.loc[by_sex['sex']==2,x]
print(a, a+1)
print(type(a))

In [ ]:
dict_ttest={} 
for x in vars:
    print(f'variable = {x}')
    mean_F = by_sex.loc[by_sex['sex']==2,x].tolist()[0]
    mean_M = by_sex.loc[by_sex['sex']==1,x].tolist()[0]
    s_F = by_sex.loc[by_sex['sex']==2,'sd_'+x].tolist()[0]
    s_M = by_sex.loc[by_sex['sex']==1,'sd_'+x].tolist()[0]
    t = (mean_F- mean_M) / (s_F**2 / n_F + s_M**2 / n_M )**0.5
    print(f't = {t}')
    dict_ttest.update({x : t})
print(dict_ttest)

Alternatively, using ttest in scipy: 

In [ ]:
import scipy.stats as stat
for x in vars:
    print(f'variable = {x}')
    group1 = oct.loc[oct['sex']==2,x].tolist() 
    group2 = oct.loc[oct['sex']==1,x].tolist()
    t,p = stat.ttest_ind(group1, group2, equal_var=False)
    print(f't = {t}, p-value = {p}')


The t-tests show that there is no siginificant gap between females and males in using cell phones for calls/browsing web. But females are more likely than to text, record videos, and use phone for social media. Males are significantly more likely to use phones for emails and music, and marginally significant (10% but not at 5% sig. level) more likely to download apps. 